In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE125158"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE125158"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE125158.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE125158.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE125158.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Development of novel diagnostic system for pancreas cancer including early stages measuring mRNA of whole blood cells"
!Series_summary	"The aim of the study is the assessment of diagnostic efficacy of extracorporeal diagnostic reagent pancreatic cancer diagnosis kit whose principle is the measurement of gene expression affected in pancreatic cancer human patients. Clinical trial registration number: UMIN000020758."
!Series_overall_design	"Development of a blood mRNA pancreatic ductal adenocarcinoma (PDAC) screening system using real-time detection PCR to detect the expression of specific genes, in order to discriminate PDAC from healthy volunteers. We performed a clinical study to assess the performance of the developed system. We collected whole blood RNA from PDAC patients and non-cancer subjects. The sensitivity of the system for PDAC diagnosis and the specificity for non-cancer volunteers were determined for the evaluation of the screening sys

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset information

# 1. Gene Expression Data Availability
# Based on the Background Information, this is a study measuring mRNA of whole blood cells
# for pancreatic cancer diagnosis, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# - Trait (Pancreatic cancer) information is at key 0 ("diagnosis")
# - Gender information is at key 2 ("gender")
# - Age information is at key 3 ("age")
trait_row = 0
gender_row = 2
age_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary (0=healthy, 1=pancreatic cancer)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower().split(": ")[-1].strip()
    if "pancreatic cancer" in value:
        return 1
    elif "healthy" in value:
        return 0
    else:
        return None

def convert_gender(value):
    """Convert gender values to binary (0=female, 1=male)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower().split(": ")[-1].strip()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age values to continuous integers"""
    if not isinstance(value, str):
        return None
    
    try:
        age = value.split(": ")[-1].strip()
        return int(age)
    except:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the result
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM3564350': [0.0, 64.0, 1.0], 'GSM3564351': [0.0, 76.0, 1.0], 'GSM3564352': [0.0, 65.0, 0.0], 'GSM3564353': [0.0, 69.0, 1.0], 'GSM3564354': [0.0, 70.0, 0.0], 'GSM3564355': [0.0, 65.0, 1.0], 'GSM3564356': [0.0, 73.0, 0.0], 'GSM3564357': [0.0, 64.0, 0.0], 'GSM3564358': [0.0, 61.0, 1.0], 'GSM3564359': [0.0, 67.0, 0.0], 'GSM3564360': [0.0, 64.0, 0.0], 'GSM3564361': [0.0, 42.0, 0.0], 'GSM3564362': [0.0, 52.0, 0.0], 'GSM3564363': [1.0, 69.0, 1.0], 'GSM3564364': [1.0, 64.0, 1.0], 'GSM3564365': [1.0, 55.0, 0.0], 'GSM3564366': [1.0, 80.0, 0.0], 'GSM3564367': [1.0, 60.0, 0.0], 'GSM3564368': [1.0, 60.0, 1.0], 'GSM3564369': [1.0, 78.0, 1.0], 'GSM3564370': [1.0, 78.0, 1.0], 'GSM3564371': [1.0, 67.0, 0.0], 'GSM3564372': [1.0, 72.0, 0.0], 'GSM3564373': [1.0, 80.0, 0.0], 'GSM3564374': [1.0, 58.0, 1.0], 'GSM3564375': [1.0, 65.0, 1.0], 'GSM3564376': [1.0, 68.0, 0.0], 'GSM3564377': [1.0, 74.0, 1.0], 'GSM3564378': [1.0, 71.0, 1.0], 'GSM3564379': [1.0, 66.0, 1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(+)eQC-39', '(+)eQC-40',
       '(+)eQC-41', '(+)eQC-42', '(-)3xSLv1', 'A_23_P100001', 'A_23_P100011',
       'A_23_P100022', 'A_23_P100056', 'A_23_P100074'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers from the previous step
# These identifiers (e.g., 'A_23_P100001') appear to be Agilent microarray probe IDs 
# which need to be mapped to human gene symbols for proper analysis
# The format A_23_P followed by numbers is typical of Agilent microarray probes

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols
probe_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Get the gene mapping dataframe with the two relevant columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene data after mapping
print("Gene data after mapping (first 10 gene symbols):")
print(gene_data.index[:10])
print(f"Shape of gene expression data: {gene_data.shape}")

# Check for NaN values percentage
nan_percentage = gene_data.isna().mean().mean() * 100
print(f"Percentage of NaN values in gene data: {nan_percentage:.2f}%")

# Save the processed gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene data after mapping (first 10 gene symbols):
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')
Shape of gene expression data: (18488, 30)
Percentage of NaN values in gene data: 0.00%


Gene expression data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE125158.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Verify we have the clinical_data from earlier steps
try:
    # Load the saved clinical data if it exists
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data from file with shape: {clinical_df.shape}")
except:
    # Use the clinical data extracted in the previous steps
    print(f"Using clinical data from previous steps with shape: {selected_clinical_df.shape}")
    clinical_df = selected_clinical_df

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Debug - print column names to identify trait column
print(f"Column names in linked_data: {linked_data.columns.tolist()}")

# Get the actual trait column name (should be the first column)
trait_col = linked_data.columns[0]
print(f"Using trait column: {trait_col}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_col)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains whole blood gene expression data for pancreatic cancer patients and healthy controls."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE125158.csv
Loaded clinical data from file with shape: (3, 30)
Linked data shape: (30, 18250)
Column names in linked_data: [0, 1, 2, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AADACL2', 'AADAT', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2', 'AARD', 'AARS1', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AASS', 'AATF', 'AATK', 'ABAT', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA12', 'ABCA13', 'ABCA2', 'ABCA3', 'ABCA4', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA8', 'ABCA9', 'ABCB1', 'ABCB10', 'ABCB11', 'ABCB4', 'ABCB5', 'ABCB6', 'ABCB7', 'ABCB8', 'ABCB9', 'ABCC1', 'ABCC10', 'ABCC11', 'ABCC12', 'ABCC13', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCC6', 'ABCC8', 'ABCC9', 'ABCD1', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCE1', 'ABCF1', 'ABCF2', 'ABCF3', 'ABCG1', 'ABCG2', 'ABCG4', 'ABCG5', 'ABCG8', 'ABHD1', 'ABHD10', 'ABHD11', 'ABHD12', 'ABHD12B', 'ABHD13', 'ABHD14A', 'ABHD14B', 'ABHD15', 'ABHD16A'

Shape after handling missing values: (30, 18250)
For the feature '0', the least common label is '0.0' with 13 occurrences. This represents 43.33% of the dataset.
The distribution of the feature '0' in this dataset is fine.



Linked data saved to ../../output/preprocess/Pancreatic_Cancer/GSE125158.csv
